In [1]:
!pip install gradio_client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.2/318.2 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.7 MB/s eta 0:00:00


First Llama 3.1 8B model prompt is not very precise, it gives out more text than we want it to! We just want jsonn with the email signature

In [ ]:
from gradio_client import Client

client = Client("vilarin/Llama-3.1-8B-Instruct")

systemPrompt = '''You are a helpful and meticulous assistant specializing in extracting structured information from text. Your primary task is to identify and extract the following entities from a given text:

* **Full Name:** The complete name of a person.
* **Email Address:** A valid email address in the format user@domain.com.
* **Company Name:** The name of an organization or business.
* **Location:** The physical address or geographic location.
* **Telephone Number:**  A phone number in any standard format.

If any of these entities are not present in the text, you should explicitly indicate that they were not found. Your response should be presented in a clear and concise format, ideally using bullet points or a similar structure to organize the extracted information.
'''

msg = '''Jane Doe, Sales & Marketing Director at ABC Company (3373 Gregory Lane, 40601 Frankfort, t: (800) 555-1133, e: jane.doe@example.com, www.example.com) emailed Mike to follow up on a project discussed during a meeting, asking if he had any updates or needed assistance. The email, inspired by professional email signature examples from blog.hubspot.com/marketing/professional-email-signatures, notes its confidentiality and requests that it be deleted if received in error. [invalid URL removed] may offer further context on similar professional communications'''

result = client.predict(
  message=msg,
  system_prompt=systemPrompt,
  temperature=0.8,
  max_new_tokens=1024,
  top_p=1,
  top_k=20,
  penalty=1.2,
  api_name="/chat"
)

print(result)

Loaded as API: https://vilarin-llama-3-1-8b-instruct.hf.space ✔
Here's the extracted information organized into bullets:

• **Full Name**: Jane Doe
• **Email Address**: jane.doe@example.com
• **Company Name**: ABC Company
• **Location**: 3373 Gregory Lane, 40601 Frankfort
• **Telephone Number**: Not formatted as requested; however, I was able to partially interpret the provided telephone number which appears to match "t: (800) 555-1133" 

Note that while the telephone number seems to conform to some known formats, there isn't enough detail for absolute identification according to specified requirements. The entity 'company website' wasn't requested but included for completeness: 
www.example.com


Even after after one-shot prompting it's not effecient

In [3]:
from gradio_client import Client

client = Client("vilarin/Llama-3.1-8B-Instruct")

system_prompt = """You are a helpful and meticulous assistant specializing in extracting structured information from text. Your primary task is to identify and extract the following entities from a given text and return them in JSON format:

{
"Full Name": "...",
"Email Address": "...",
"Company Name": "...",
"Location": "...",
"Telephone Number": "..."
}

If an entity is not found, its value should be set to "Not Found".

Prioritize accuracy and completeness when extracting information. If you are unsure about a value, indicate "Not Found" rather than guessing.
* **Location:** This could be a full street address, city and state, or even just a country.
* **Telephone Number:** Include country codes if present, and format as +[Country Code] [Number].


Example:
Text: "John Doe (john.doe@email.com) called from ACME Corp. in New York (123) 456-7890."

Response:
{
"Full Name": "John Doe",
"Email Address": "john.doe@email.com",
"Company Name": "ACME Corp.",
"Location": "New York",
"Telephone Number": "(123) 456-7890"
}
"""

msg = '''Jane Doe, Sales & Marketing Director at ABC Company (3373 Gregory Lane, 40601 Frankfort, t: (800) 555-1133, e: jane.doe@example.com, www.example.com) emailed Mike to follow up on a project discussed during a meeting, asking if he had any updates or needed assistance. The email, inspired by professional email signature examples from blog.hubspot.com/marketing/professional-email-signatures, notes its confidentiality and requests that it be deleted if received in error. [invalid URL removed] may offer further context on similar professional communications'''


result = client.predict(
message=msg,
system_prompt=system_prompt,
temperature=0.2,
max_new_tokens=1024,
top_p=0.95,
api_name="/chat"
)

print(result)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loaded as API: https://vilarin-llama-3-1-8b-instruct.hf.space ✔
Here's the extracted data in JSON format:

```json
{
    "Full Name": "Jane Doe",
    "Email Address": "jane.doe@example.com",
    "Company Name": "ABC Company",
    "Location": "Frankfort, 40601",
    "Telephone Number": "+1 (800) 555-1133"
}
```

Note that I've formatted the telephone number with a plus sign (+) for international dialing and included the country code '1' assuming it's a US-based phone number. Also, since there was no explicit mention of Jane's title being part of her name, I didn't include it in the Full Name field.

I did not attempt to parse the invalid URL mentioned in the original message because it does not contain relevant contact information.


Nemo-Mistral is more accurate an Robust, parsing the same prompt i parsed to llama and it get it right

In [ ]:
client = Client("vilarin/Mistral-Nemo")

jane_doe_text = "Jane Doe, Sales & Marketing Director at ABC Company (3373 Gregory Lane, 40601 Frankfort, t: (800) 555-1133, e: jane.doe@example.com, www.example.com) emailed Mike to follow up on a project discussed during a meeting, asking if he had any updates or needed assistance. The email, inspired by professional email signature examples from blog.hubspot.com/marketing/professional-email-signatures, notes its confidentiality and requests that it be deleted if received in error. [invalid URL removed] may offer further context on similar professional communications"

msg = f"""You are a helpful and meticulous assistant specializing in extracting structured information from text. Your primary task is to identify and extract the following entities from this text:

'{jane_doe_text}'

* **Full Name:** The complete name of a person.
* **Email Address:** A valid email address in the format user@domain.com.
* **Company Name:** The name of an organization or business.
* **Location:** The physical address or geographic location.
* **Telephone Number:** A phone number in any standard format.

If any of these entities are not present in the text, you should explicitly indicate that they were not found. Your response should be presented in a clear and concise format, ideally using bullet points or a similar structure to organize the extracted information."""


result = client.predict(
  message=msg,
  temperature=0.3,
  max_new_tokens=1024,
  top_p=1,
  top_k=20,
  penalty=1.2,
  api_name="/chat"
)
print(result)

Loaded as API: https://vilarin-mistral-nemo.hf.space ✔
- **Full Name:** Jane Doe
- **Email Address:** jane.doe@example.com
- **Company Name:** ABC Company
- **Location:** 3373 Gregory Lane, 40601 Frankfort
- **Telephone Number:** (800) 555-1133
- **Website:** www.example.com


Final Prompts and wrapper to filter the output of llama 3.1 8B

In [42]:
from gradio_client import Client

client = Client("vilarin/Mistral-Nemo")

user_prompt = """Jane Doe, Sales & Marketing Director at ABC Company (3373 Gregory Lane, 40601 Frankfort, t: (800) 555-1133, e: jane.doe@example.com, www.example.com) emailed Mike to follow up on a project discussed during a meeting, asking if he had any updates or needed assistance. The email, inspired by professional email signature examples from blog.hubspot.com/marketing/professional-email-signatures, notes its confidentiality and requests that it be deleted if received in error. [invalid URL removed] may offer further context on similar professional communications"""

msg = f"""You are a helpful and meticulous assistant specializing in extracting structured information from text. Your primary task is to identify and extract the following entities from a given text and return them in JSON format:

{{
"Full Name": "...",
"Email Address": "...",
"Company Name": "...",
"Location": "...",
"Telephone Number": "..."
}}

If an entity is not found, its value should be set to "Not Found".

Prioritize accuracy and completeness when extracting information. If you are unsure about a value, indicate "Not Found" rather than guessing.
* **Location:** This could be a full street address, city and state, or even just a country.
* **Telephone Number:** Include country codes if present, and format as +[Country Code] [Number].


Example:
Text: "John Doe (john.doe@email.com) called from ACME Corp. in New York (123) 456-7890."

Response:
{{
"Full Name": "John Doe",
"Email Address": "john.doe@email.com",
"Company Name": "ACME Corp.",
"Location": "New York",
"Telephone Number": "(123) 456-7890"
}}

Now extract information from this text:

'{user_prompt}'
"""

# Make the API call
result = client.predict(
  message=msg,
  temperature=0.2,
  max_new_tokens=1024,
  top_p=0.95,
  api_name="/chat"
)

print(result)

Loaded as API: https://vilarin-mistral-nemo.hf.space ✔
{
"Full Name": "Jane Doe",
"Email Address": "jane.doe@example.com",
"Company Name": "ABC Company",
"Location": "3373 Gregory Lane, 40601 Frankfort",
"Telephone Number": "(800) 555-1133"
}


Building a output wrapper for the llama model

In [56]:
import re

def filterOutput(result_text):
  pattern = r"\{([^{}]*)\}"
  matches = re.findall(pattern, result_text)
  cleaned_string = matches[0].strip().replace('\n  ', '')
  data_dict = json.loads(f"{{{cleaned_string}}}")
  formatted_output = "\n".join([f" **{key}: {value}" for key, value in data_dict.items()])
  return formatted_output

In [33]:
import json

def bulletPoints(data_string):
  cleaned_string = data_string[0].strip().replace('\n  ', '')
  data_dict = json.loads(f"{{{cleaned_string}}}")
  for key, value in data_dict.items():
      ret=f"- {key}: {value}"
      return ret

- Full Name: Jane Doe
- Title: Sales & Marketing Director
- Company Name: ABC Company
- Address: 3373 Gregory Lane, 40601 Frankfort
- Phone Number: (800) 555-1133
- Email Address: jane.doe@example.com
- Website: www.example.com


In [58]:
from gradio_client import Client

client = Client("vilarin/Llama-3.1-8B-Instruct")

user_prompt = """Jane Doe, Sales & Marketing Director at ABC Company (3373 Gregory Lane, 40601 Frankfort, t: (800) 555-1133, e: jane.doe@example.com, www.example.com) emailed Mike to follow up on a project discussed during a meeting, asking if he had any updates or needed assistance. The email, inspired by professional email signature examples from blog.hubspot.com/marketing/professional-email-signatures, notes its confidentiality and requests that it be deleted if received in error. [invalid URL removed] may offer further context on similar professional communications"""

msg = f"""Extract ONLY the email signature from the following text and return it in JSON format. If there is no signature, return "Not Found".
{{
"Full Name": "...",
"Email Address": "...",
"Company Name": "...",
"Location": "...",
"Telephone Number": "..."
}}

If an entity is not found, its value should be set to "Not Found".

Prioritize accuracy and completeness when extracting information. If you are unsure about a value, indicate "Not Found" rather than guessing.
* **Location:** This could be a full street address, city and state, or even just a country.
* **Telephone Number:** Include country codes if present, and format as +[Country Code] [Number].


Example:
Text: "John Doe (john.doe@email.com) called from ACME Corp. in New York (123) 456-7890."

Response:
{{
"Full Name": "John Doe",
"Email Address": "john.doe@acmecorp.com",
"Company Name": "ACME Corp.",
"Location": "Anytown, CA 12345",
"Telephone Number": "(123) 456-7890"
}}

Now extract the signature from this text:

'{user_prompt}'
"""

result = client.predict(
  message=msg,
  temperature=0.2,
  max_new_tokens=1024,
  top_p=0.95,
  api_name="/chat"
)

print(filterOutput(result))

Loaded as API: https://vilarin-llama-3-1-8b-instruct.hf.space ✔
 **Full Name: Jane Doe
 **Title: Sales & Marketing Director
 **Company Name: ABC Company
 **Address: 3373 Gregory Lane, 40601 Frankfort
 **Phone Number: (800) 555-1133
 **Email Address: jane.doe@example.com
 **Website: www.example.com
